In [5]:
from land_frontend import movement
from land_frontend import clean_movement
import dijkstra_methods as dk
from datetime import datetime
from datetime import timedelta
import time

#helping method to determine if a patrol is hit
def isin(pattern, seq):
    for i in range(len(seq) - len(pattern) + 1):
        if seq[i:i+len(pattern)] == pattern:
            return True
    return False
#helping method for determining if a province has a holdfast
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


def patrol_sweep(speed,start,end,sdatetime='now'):
    #initialise possible patrol list
    patrol_list = []
    #read patrol database
    filename = './sample_patrols.txt'
    with open(filename) as fhandle:
        for line in fhandle:
            #print(line)
            p1,p2,desc = line.split('/')
            patrol_list.append([p1,p2,desc])
#print(patrol_list)
    #set datetime
    if sdatetime=='now':
        sdatetime = datetime.now()
    elif isinstance(sdatetime,datetime):
        pass
    else:
        sdatetime = datetime.strptime(sdatetime,'%d/%m/%y %H:%M:%S')
    path,time,dlta = movement(speed,start,end)
    print('checking for patrols--------------------------------------------------------')
    for patrol in patrol_list:
        desc = patrol[2]
        coordinates = patrol[0:2]
        #cut one tile patrols to the appropriate length
        if coordinates[1]=='0':
            coordinates=[coordinates[0]]
        #catch forward patrols
        if isin(coordinates,list(path)): 
            #calculate time delta and end time for a tripped patrol
            pth,endtime,dlta = clean_movement(speed,start,str(coordinates[-1]),sdatetime)
            print('patrol tripped at ',coordinates,' after ',str(dlta),' hours, at ',endtime)
            print(desc)
        #catch reverse patrols
        elif isin(coordinates[::-1],list(path)):
            #calculate time delta and end time for a tripped patrol
            pth,endtime,dlta = clean_movement(speed,start,str(coordinates[-1]),sdatetime)
            print('patrol tripped at ',coordinates,' after ',str(dlta),' hours, at ',endtime)
            print(desc)
    #holdfast detections
    for province in list(path):
        if not has_numbers(province):
            desc= 'Holdfast detection at '+province
            pth,endtime,dlta = clean_movement(speed,start,province,sdatetime)
            print('patrol tripped at ',province,' after ',str(dlta),' hours, at ',endtime)
            
patrol_sweep(10,'CasterlyRock','KingsLanding')

Total hours of travel is  15.6
end time is:  2023-06-14 15:14:49.824596
deque(['CasterlyRock', 'W15', 'W20', 'W21', 'DeepDen', 'W22', 'Riverspring', 'R7', 'R8', 'R9', 'C25', 'C26', 'C14', 'KingsLanding'])
checking for patrols--------------------------------------------------------
patrol tripped at  CasterlyRock  after  0.0  hours, at  2023-06-13 23:38:49.824593
patrol tripped at  DeepDen  after  4.8  hours, at  2023-06-14 04:26:49.824593
patrol tripped at  Riverspring  after  7.2  hours, at  2023-06-14 06:50:49.824593
patrol tripped at  KingsLanding  after  15.6  hours, at  2023-06-14 15:14:49.824593
